# Globals

In [14]:
rowsASIC = 192
colsASIC = 384

# Lorenzo helper functions for data analysis

In [19]:
# Filter bad pixels
def filter_bad_pixels(data, ratio):
    """
    Filters bad pixels in the input data based on noise level.

    Parameters:
        data (numpy.ndarray): Input data array of shape (rows, columns, frames).
        ratio (float): Threshold ratio to identify bad pixels.

    Returns:
        tuple: A tuple containing the filtered data array and the number of identified bad pixels.
    """
    # Create a copy of the input data array
    data_copy = data.copy()

    # Calculate noise for each pixel
    noise = np.std(data_copy, axis=2)
    noise_median = np.median(noise)

    # Identify and replace bad pixels
    mask = noise > ratio * noise_median
    data_copy[mask] = np.median(data_copy)

    num_bad_pixels = np.sum(mask)

    return data_copy, num_bad_pixels


# Reject outliers
def rejOutliers(data, m=2):
    return data[abs(data - np.mean(data)) < m * np.std(data)]

# Correlation matrix
def calculate_correlation(data):

    corr_mat   = np.empty([np.shape(data)[0],np.shape(data)[1]])
    
    for row, col in np.ndindex(np.shape(data)[0],np.shape(data)[1]):
        # Correlation coefficients
        corr_mat[row,col] = np.corrcoef(data[row,col,:], data[ref_row, ref_col,:])[0,1]

    corr_mat[ref_row,ref_col] = np.average(corr_mat)
    return corr_mat



# # Get Average Dark value over rows/cols
# def getMedianDark(data):
#     return np.median(np.median(data,axis=(0,1)))

# # Charge injection ramp
# def ChInjRamp(dataDebug, numberOfTriggers=3, Min=100, Max=150, numSteps=3):

#     ChInjValues = np.linspace(start=Min, stop=Max, num=numSteps)
#     PixelOutValues = np.empty(np.shape(ChInjValues))
      
#     for i, PulserValue in enumerate(ChInjValues):
#         # Get frames but look only at ref pixel
#         data_temp = readoutFrames(dataDebug, numberOfTriggers,chargeInj=PulserValue)[ref_row,ref_col,:]
#         # Get median across frames and add to results array
#         data_temp = np.median(data_temp)
#         PixelOutValues[i] = data_temp
    
#     return ChInjValues, PixelOutValues
  
# Mask gain bit    
def clearB16(value):
    return value & ~(1 << 15)    

# Better functions !

This functions performs a Charge Injection Ramp.

In [13]:
# Charge injection ramp
def chargeInjRamp(dataDebug, AsicNum=1, Min=100, Max=150, numSteps=3, numSamples=1):

    ChInjValues = np.linspace(start=Min, stop=Max, num=numSteps)
    
    # Create an empty 3D array, coordinates [rows,cols,injValue]
    PixelOutValues = np.empty([rowsASIC, colsASIC, len(ChInjValues)])
      
    for i, PulserValue in enumerate(ChInjValues):
        # Get frames but look only at ref pixel
        data_temp = readoutFrames(dataDebug, AsicNum=AsicNum, numberOfTriggers=numSamples, chargeInj=PulserValue)
        # Get median across frames and add to results array
        data_temp = np.median(data_temp, axis=2)
        PixelOutValues[:,:,i] = data_temp
        print("Injecting ramp point {}/{}".format(i, np.shape(ChInjValues)[0]), end='\r')
    
    return ChInjValues, PixelOutValues

Function to program the charge injection pattern.

In [2]:
# Configure charge injection
def chargeInjection(firstCol=140,LastCol=160,PulserValue=100,AsicNum=3):
    APP.prepareChargeInjection(AsicNum, firstCol, LastCol, PulserValue)


Function to readout frames with SW trigger.
If chargeInj is specified, the chargeInjection procedure is called before each frame (we need to program the pixels each time we take a frame).

In [7]:
# Readout frame(s)

def readoutFrames(dataDebug, AsicNum=0, numberOfTriggers=1, chargeInj=False):
    dataDebug.cleanData()

    for TrigNum in range(numberOfTriggers):
        if (chargeInj != False):
            chargeInjection(PulserValue=chargeInj, AsicNum=AsicNum)
        root.Trigger()
        time.sleep(0.01)

    # Although the triggers above may have finished, descrambling takes time
    while (numberOfTriggers != dataDebug.getData().shape[2]) :
        time.sleep(0.1)

    data = dataDebug.getData()
    dataDebug.cleanData()
    return data


# Timing functions

In [ ]:
# helper functions
# clock speed is 168MHz. 1 tick is 0.006 us
def setSR0(width, delay) :
    REGCTRL.SR0Delay1.set(delay)
    REGCTRL.SR0Width1.set(width)
    print(f'SR0 Width set to {REGCTRL.SR0Width1_us.get():.3f}uS')
    print(f'SR0 Delay set to {REGCTRL.SR0Delay_us.get():.3f}uS')
    
def setAcq1(width, delay) :
    REGCTRL.AcqDelay1.set(delay)
    REGCTRL.AcqWidth1.set(width)
    print(f'Acq1 Width set to {REGCTRL.AcqWidth1_us.get():.3f}uS')
    print(f'Acq1 Delay set to {REGCTRL.AcqDelay1_us.get():.3f}uS')
    
def setAcq2(width, delay) :
    REGCTRL.AcqDelay2.set(delay)
    REGCTRL.AcqWidth2.set(width)
    print(f'Acq2 Width set to {REGCTRL.AcqWidth2_us.get():.3f}uS')
    print(f'Acq2 Delay set to {REGCTRL.AcqDelay2_us.get():.3f}uS')    

def setR0(width, delay) :
    REGCTRL.R0Delay.set(delay)
    REGCTRL.R0Width.set(width)
    print(f'R0 Width set to {REGCTRL.R0Width_us.get():.3f}uS')
    print(f'R0 Delay set to {REGCTRL.R0Delay_us.get():.3f}uS') 

    
def setSync(width, delay) :
    REGCTRL.SyncDelay.set(delay)
    REGCTRL.SyncWidth.set(width)
    print(f'Sync Delay set to {REGCTRL.SyncDelay_us.get():.3f}uS')     

# Best settings
These are still w.i.p.

In [4]:
def setBestSettings(REGCTRL, ASIC0, ASIC1, ASIC2, ASIC3):

    intTime = 560 # 560 = 3.6uS

    setR0(100+intTime*2, 200)
    setAcq1(intTime, intTime+200)

    # Print human-readable values
    print(f'-------------------------')
    baseline_int_time = REGCTRL.AcqDelay1_us.get() - REGCTRL.R0Delay_us.get()
    integration_time = REGCTRL.AcqWidth1_us.get()
    print(f'Baseline time:    {baseline_int_time} uS')
    print(f'Integration time: {integration_time} uS')
    print(f'Sampling done at: {REGCTRL.AcqWidth1_us.get()+REGCTRL.AcqDelay1_us.get()} uS')
    print(f'-------------------------')
        
    ASIC1.pipoclk_delay_row0.set(15)
    ASIC1.pipoclk_delay_row1.set(8)
    ASIC1.pipoclk_delay_row2.set(8)
    ASIC1.pipoclk_delay_row3.set(8)

    ASIC2.pipoclk_delay_row0.set(5)
    ASIC2.pipoclk_delay_row1.set(5)
    ASIC2.pipoclk_delay_row2.set(5)
    ASIC2.pipoclk_delay_row3.set(5)
        
    print('ASIC configured')

The settings below are for the analog/ADC, and are the same for all ASIC. This function should be done for all ASICs.

In [17]:
def setAnalogSettings(ASIC):
    
    # Set ASIC registers
    ASIC.mTest.set(False)
    ASIC.CompTH_ePixM.set(0)
    ASIC.Precharge_DAC_ePixM.set(45)
    
    # Optimal bias settings
    ASIC.RefinN.set(0)
    ASIC.RefinP.set(0)
    
    # ADC settings
    ASIC.RefGenB.set(1)    #2 bits
    ASIC.RefGenC.set(1)    #2 bits
    ASIC.Ref_gen_d.set(3) # Set 'common-mode' voltage of S/H stage
    ASIC.S2D_1_b.set(0)    #3 bits
    ASIC.shvc_DAC.set(30)  #6 bits
    ASIC.S2dDacBias.set(3)
    ASIC.DHg.set(True)

# Progressive colormap

In [8]:
from matplotlib import cm

def genColors(num):
    return cm.copper(np.linspace(0, 1, num))

# Linear fit 
- more weight at beginning for HG
- more weight at mid for LG

In [5]:
def linearFitHG(x, y):
    """
    Apply a linear fit with more weight on the initial points.

    Parameters:
    x (numpy.array): The independent variable.
    y (numpy.array): The dependent variable.

    Returns:
    tuple: A tuple containing the residuals, slope, and intercept.
    """
    # Generate weights: Here we use an exponentially decreasing weight.
    # You can adjust the rate of decrease as per your requirements.
    weights = np.exp(-np.linspace(0, 5, len(y)))
    
    # Apply the weighted linear fit
    coeffs = np.polyfit(x, y, deg=1, w=weights)
    slope, intercept = coeffs
    
    # Calculate the fitted values
    y_fit = np.polyval(coeffs, x)
    
    # Calculate the residuals
    residuals = y - y_fit
    
    return residuals, slope, intercept

In [7]:
import numpy as np

def linearFitLG(x, y):
    """
    Apply a linear fit with more weight on the initial points.

    Parameters:
    x (numpy.array): The independent variable.
    y (numpy.array): The dependent variable.

    Returns:
    tuple: A tuple containing the residuals, slope, and intercept.
    """
    # Generate weights: Here we use an exponentially decreasing weight.
    # You can adjust the rate of decrease as per your requirements.
    n = len(y)
    weights = np.exp(-((np.arange(n) - (n - 1) / 2) ** 2) / (2 * ((n / 10) ** 2)))

    
    # Apply the weighted linear fit
    coeffs = np.polyfit(x, y, deg=1, w=weights)
    slope, intercept = coeffs
    
    # Calculate the fitted values
    y_fit = np.polyval(coeffs, x)
    
    # Calculate the residuals
    residuals = y - y_fit
    
    return residuals, slope, intercept